## Computational ressources

In [ ]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

In [ ]:
cluster.scale(102) 


In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

## Librairies

In [ ]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen

import matplotlib.pyplot as plt


## Importing the data : SSH in run eNATL60-BLBT02 for FMA months

In [ ]:
tfilenames = []

for month in ['02','03','04']:
    if month in ['12','01','02','03','04','05','06']:
        case='BLBT02X'
    else:
        case='BLBT02'

    if month in ['07','08','09','10','11','12']:
        year='2009'
    else:
        year='2010'
    
    data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-'+str(case)+'-S/'
    tfilenames_month = sorted(glob.glob(data_dir+'*/eNATL60-'+str(case)+'_1h_*_gridT-2D_'+str(year)+str(month)+'??-'+str(year)+str(month)+'??.nc'))
    tfilenames=tfilenames+tfilenames_month

In [ ]:
ds=xr.open_mfdataset(tfilenames,chunks={'x':1000,'y':1000,'time_counter':1})['sossheig']
    

In [ ]:
ds.nbytes/1e9

In [ ]:
ds

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
ds[0].plot()

## SSH anomalies

In [ ]:
ssh_c=ds-ds.mean()

## Region of interest

In [ ]:
ssh_c_1pt=ssh_c[:,1000,1000]

## FFT decomposition and power spectral density computation

In [ ]:
NPADDING = 2
SPtime = xfft.fft(ssh_c, dim='time_counter', dx=1., detrend='mean',nfft=ssh_c.shape[0]*NPADDING,tapering=True)
MEANPSD=xfft.psd(SPtime).load()
freqs=MEANPSD.f_time_counter.values
freqs = freqs[NPADDING::] 


In [ ]:
xmin=-3
xmax=0

title=regna

fig = plt.figure(figsize=(6, 6), dpi=dpifig,facecolor='white')
ax = fig.add_subplot(111)

ax.loglog(freqs,MEANPSD[NPADDING::])
#ax.set_xlim(1e-3,1)
